In [24]:
%matplotlib inline

import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler

In [40]:
path = '/Users/kia/Desktop/SYS 6016/HW/hw4/'
file = 'train.txt'
corpus = pd.read_csv(path+file, sep=r'\s{2,}',header=None)
corpus.shape

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


(72193, 1)

In [41]:
train_corpus = corpus.head(100)

In [42]:
train_corpus = train_corpus.values.T.tolist()

In [54]:
#convert corpus to list
reviews = list(train_corpus)
reviews = [str(review) for review in reviews]

In [88]:
#split into sentences
lines = [re.split('[?.!]', review.lower()) for review in reviews]
#strip whitespace
lines = [[sentence.strip(' ') for sentence in sentences] for sentences in lines]
#combine sentences from different documents
sentences = [j for i in lines for j in i]
#split sentences into words
words = [re.split(' ', sentence) for sentence in sentences]
words = str(words).split()
words = [''.join(e for e in string if e.isalnum()) for string in words]

In [89]:
sequences = list()
for i in range(1, len(words)):
	sequence = words[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 21024


In [91]:
sequences[1]

['brought', 'two']